In [1]:
import pandas as pd
from helpers import load_all_datasets, compose
import random
from transformers import LlamaTokenizerFast

SAFE_FILE = "validation_pe2_gpt2_input"

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_, _, _, _, cbr_database, _, validation = load_all_datasets()

In [3]:
tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

In [4]:

inputs = []
pe_recipes = []
target_recipe = []

for i, e in validation.iterrows():

    while True:
        recipe_ixs = [random.randint(0, len(cbr_database)-1) for _ in range(5)]
        gen_input = compose(cbr_database.iloc[recipe_ixs], e)
        if len(tokenizer.encode(gen_input)) <= 850: 
            break
        else:
            for ii in list(range(6, -1, -1)):
                gen_input = compose(cbr_database.iloc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], e)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(5, -1, -1)):
                gen_input = compose(cbr_database.iloc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], e)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        
    pe_recipes.append(list(cbr_database.iloc[recipe_ixs].id.values))
    target_recipe.append(e["id"])
    inputs.append(gen_input)



1 - 805
1 - 745
1 - 849
1 - 762
1 - 707
1 - 639
1 - 828
1 - 693
1 - 779
1 - 750
1 - 679
1 - 691
1 - 786
1 - 789
1 - 623
1 - 646
1 - 686
1 - 751
1 - 743
1 - 748
1 - 806
1 - 630
1 - 814
1 - 703
1 - 690
1 - 790
1 - 717
1 - 679
1 - 828
1 - 809
1 - 715
1 - 677
1 - 821
1 - 740
1 - 777
1 - 785
1 - 743
1 - 766
1 - 795
1 - 816
1 - 727
1 - 794
1 - 717
1 - 847
1 - 735
1 - 786
1 - 715
1 - 774
1 - 751
1 - 804
1 - 847
1 - 695
1 - 697
1 - 808
1 - 716
1 - 775
1 - 679
1 - 774
1 - 745
1 - 813
1 - 846
1 - 684
1 - 790
1 - 746
1 - 714
1 - 747
1 - 800
1 - 737
1 - 769
1 - 689
1 - 778
1 - 762
1 - 761
1 - 736
1 - 781
1 - 738
1 - 745
1 - 846
1 - 717
1 - 693
1 - 739
1 - 831
1 - 802
1 - 799
1 - 742
1 - 762
1 - 794
1 - 686
1 - 734
1 - 731
1 - 796
1 - 744
1 - 605
1 - 803
1 - 680
1 - 818
1 - 828
1 - 740
1 - 755
1 - 768
1 - 734
1 - 724
1 - 740
1 - 799
1 - 706
1 - 840
1 - 716
1 - 838
1 - 758
1 - 689
1 - 681
1 - 685
1 - 848
1 - 773
1 - 774
1 - 669
1 - 704
1 - 754
1 - 805
1 - 731
1 - 792
1 - 738
1 - 708
1 - 746
1 - 706


In [5]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{SAFE_FILE}.csv", index=False)


In [6]:
from collections import Counter


Counter(df.input.apply(lambda x: x.split("name:")).apply(lambda x: len(x)))

Counter({6: 1207, 7: 918})